# Claim Extraction Experiment

In [15]:
import os
from google import genai
from google.genai import types
from src.utils.loader_googlefile import load_video_file_from_metadata, save_video_file_metadata

import time


In [4]:
client = genai.Client()

### Video Upload

In [13]:
## fake video downloaded
video_path = "../data/socialdf/socialdf_vids/vids/C96nZIGIb_v/2024-07-27_06-24-04_UTC.mp4"
with open(video_path, 'rb') as f:
    video_file = client.files.upload(
        file=f,
        config=types.UploadFileConfig(mime_type="video/mp4")
    )

print(f"Uploaded file: {video_file.name}")

# Wait for the file to be processed
print("Waiting for video processing...")
while video_file.state == "PROCESSING":
    time.sleep(2)
    video_file = client.files.get(name=video_file.name)

if video_file.state == "FAILED":
    raise ValueError("Video processing failed")

print("Video processing complete!")

Uploaded file: files/2umem6nzmg3o
Waiting for video processing...
Video processing complete!


In [15]:
prompt = """
    Por favor faça uma extração abrangente deste vídeo incluindo:
    - Principais tópicos discutidos
    - Participantes do vídeo e tom do discurso
    """
    
# Generate the summary
print("Generating summary...")
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        types.Part.from_uri(file_uri=video_file.uri, mime_type=video_file.mime_type),
        prompt
    ]
)


Generating summary...


In [18]:
print(response.text)

Aqui está uma extração abrangente do vídeo:

**Principais Tópicos Discutidos:**
*   **Jogo de Azar (Gambling):** O tema central é o jogo de azar, especificamente a pressão social e os avisos que Steve Harvey (e, por extensão, o espectador) recebe sobre o assunto.
*   **Percepções Sociais sobre o Jogo de Azar:** O vídeo explora a visão comum de que o jogo de azar é prejudicial, leva à perda de dinheiro e ao vício.
*   **Experiência Pessoal vs. Conselhos Comuns:** Steve Harvey contrasta os conselhos negativos que recebe com sua própria experiência, sugerindo que ele não vê o jogo de azar como algo inerentemente ruim.
*   **A Marca Rollbit:** A presença constante do logotipo "ROLLBIT" na parte inferior do vídeo sugere uma promoção ou patrocínio de uma plataforma de cripto-jogos/apostas.

**Pontos-Chave ou Destaques:**
*   Steve Harvey relata que as pessoas o aconselham a parar de jogar, alertando que ele "perderá todo o seu dinheiro e se tornará viciado".
*   Ele desconsidera esses avisos